# Intensity-based segmentation of the eGFP/mCherry selected MEF cell

Here, we use the generated intensity image and use standard image segmentation techniques like Median or Gaussian filtering, Li- or Otsu-based thresholding to separate the three regions of interest: cytoplasm, vesicles and nucleus.
At the end these segmentations represent binary masks, which are filled with "1/TRUE" in the region of interest and "0/FALSE" outside.

In [1]:
# Importing of all required functions from scikit-image
from skimage.filters import threshold_otsu, median, gaussian, threshold_li
from skimage import io, img_as_ubyte
from skimage.morphology import disk, area_closing, remove_small_objects, dilation
from skimage.util import invert
from scipy.ndimage import binary_fill_holes
import matplotlib.pyplot as plt
import numpy as np

# Note: The segmentation is performed on the sum of all channels (green/red) and time windows (prompt/delay)

## Select cell area

We start with the segmentation of the cell in the center of the image. Here, first a median filtering with a disk size of 3 pixel is applied. This is followed by thresholding using the method of Otsu. However, for later frames direct usage of the Otsu threshold selected too much of background, thus we slightly increase the threshold by multiplying with 1.03 (Otsu_scaling_factor) In the final steps, first small holes inside the cytoplasm area are closed, followed by the removal of small detected areas outside the main cell. Here, a minimal size threshold of 10’000 pixel is applied.

In [2]:
# Load the intensity sum data
filename = "./FLIM/mGBP_DA.ht3_sum.tif"
img = img_as_ubyte(io.imread(filename))

/opt/tljh/user/lib/python3.9/site-packages/skimage/util/dtype.py:550: UserWarning: Downcasting uint16 to uint8 without scaling because max value 63 fits in uint8
  return _convert(image, np.uint8, force_copy)


In [3]:
# Segmentation parameter
disk_size = 3  # smoothing radius in pixel
minimal_size = 10000  # give minimal cytoplasm size in pixel
scaling_factor = 1.03  # increase Otsu threshold by multiplication with this factor

In [4]:
# Loop over all frames:
# (1) Median filtering with a radius of 3 pixel
# (2) Otsu thresholding, here we increase the automatic determined value by a factor of 1.03
# (3) Fill holes in the binary mask
# (4) Remove detected areas outside the cytoplasm by a size filter

binary_img = []
for image in range(img.shape[0]):
    input_img = img[image,:,:]
    median_img = median(input_img, disk(disk_size))
    thresh_int = threshold_otsu(median_img)  # Determine Otsu threshold for each frame
    binary_temp = median_img >= thresh_int*scaling_factor  # Intensities larger than threshold = TRUE /1
    bool_img = binary_fill_holes(binary_temp)
    segmentation_filled = np.copy(bool_img.astype(np.uint8))
    small_aggregates_removed = area_closing(invert(segmentation_filled), area_threshold=minimal_size)
    final_segmentation = invert(small_aggregates_removed)
    binary_img.append(final_segmentation)

In [5]:
# Save processed image tif series of frames
# return image is an 8-bit image filled with 0's and 1's
cell_mask = (np.array(binary_img, dtype=np.ubyte))
io.imsave(filename + '_binary_cell.tif', cell_mask, check_contrast=False)

## Nucleus

Next, the nucleus is segmented. In contrast to the cytoplasm, here only the green intensity image is used as input as here the contrast between cytoplasm/vesicles region and nucleus seemed largest. Please note that the nucleus can be best seen as “region in cytoplasm but without vesicles”. The segmentation of the nucleus thus proves to be a challenge. We first apply a Gaussian smoothing with a radius of 2 sigma, followed by thresholding using the method of Li. The thresholded image is dilated, holes inside the nucleus filled and small areas outside removed. Next all left-overs outside the cytoplasmic (i.e. body of the cell) are removed by multiplying with the generated cytoplasm mask, followed by another round of dilation to make the nucleus “smoother”. Finally, as the nucleus is characterized by being outside the way of the diffusing vesicles, we average the such obtained nucleus and generate an average nucleus by summing over all 400 frames and threshold this projection using the method of Otsu.

In [6]:
# Define the parameter
# Here we only use the green channel for segmentation
input_nuc = io.imread("./FLIM/mGBP_DA.ht3_green.tif")
sigma = 2  # radius for Gaussian filtering
smoothing_radius = 2  # for dilation operation
minimal_size_nuc = 1000  # minimal area size (smaller areas will be removed)
minimal_nucleus_size = 2000 # minimal nucleus size

In [7]:
# Loop over the image to process each frame
# (1) Gaussian smoothing with a radius of 2 pixel
# (2) Li thresholding
# (3) Dilation of binary mask with smoothing radius of 2 pixel
# (4) Fill small holes
# (5) Remove small areas outside the nucleus
# (6) Combine with cytoplasm by multiplication of binary images
# (7) Remove smaller areas at the cytoplasm border
# (8) Dilate nucleus to make more "smooth"

binary_img_nucleus = []
for image in range(input_nuc.shape[0]):
    input_img = input_nuc[image,:,:]
    input_img_cytoplasm = cell_mask[image,:,:]
    gaussian_img = gaussian(input_img, sigma=sigma)
    li_threshold = threshold_li(gaussian_img)
    thresholded_image = gaussian_img > li_threshold
    dilated_segmentation = dilation(thresholded_image, disk(smoothing_radius))
    bool_img = binary_fill_holes(dilated_segmentation)
    segmentation_filled = np.copy(bool_img.astype(np.uint8))
    small_areas_removed = area_closing(invert(segmentation_filled), area_threshold=minimal_size_nuc)
    combination = np.zeros(small_areas_removed.shape, dtype=np.uint)
    combination = input_img_cytoplasm * (small_areas_removed - 254)
    cleaned_combination = area_closing(invert(combination), area_threshold=minimal_size_nuc)
    dilated_combination = dilation(invert(cleaned_combination), disk(smoothing_radius))
    binary_img_nucleus.append(dilated_combination)

In [8]:
# Save individual frames with detected "nucleus"
io.imsave(filename + '_binary_nucleus.tif', np.array(binary_img_nucleus, dtype=np.ubyte), check_contrast=False)

# Average over all frames to get a smoother nucleus
averaged_img = np.array(np.sum(binary_img_nucleus, axis=0), dtype=int)
thresh = threshold_otsu(averaged_img) # Apply Otsu thresholding
thresholded_img = averaged_img > thresh
# Remove smaller stuff
avg_nucleus = remove_small_objects(thresholded_img, min_size=minimal_nucleus_size)
# Save average nucleus
io.imsave(filename + '_avg_nucleus.tif', np.array(avg_nucleus, dtype=np.ubyte), check_contrast=False)

## Vesicle-like structures (VLS)

The vesicle-like structures (VLS) are very bright regions in the cells. Here, first a Gaussian smoothing with a size of 1 sigma is applied, followed by thresholding using the method of Otsu. Finally, objects with a size smaller than 5 pixels are removed.
Furthermore, we remove all VLS from our ROI which belong to a neighboring cell or seem to be inside the nucleus (above or below).

In [9]:
# set the parameter
sigma_VLS = 1  # radius for Gaussian filtering
min_vesicle_size = 5  # give minimal vesicle size in pixel

In [16]:
# Loop over all frames
# (1) Gaussian smoothing with a radius of 1 pixel
# (2) Otsu thresholding
# (3) Remove spots smaller than 5 pixel

binary_img_VLS = []
for image in range(img.shape[0]):
    input_img = img[image,:,:]
    gaussian_img = gaussian(input_img, sigma=sigma_VLS)
    thresh_int = threshold_otsu(gaussian_img)  # Determine Otsu threshold for each frame
    binary_temp = gaussian_img >= thresh_int  # Intensities larger than threshold = TRUE /1
    cleaned_image = remove_small_objects(binary_temp, min_vesicle_size) # Remove small
    binary_img_VLS.append(cleaned_image)

# Save processed image tif series of frames
VLS_mask = (np.array(binary_img_VLS, dtype=np.ubyte))

# Limit vesicles to area of the selected cell and outside the nucleus
VLS_in_cell = VLS_mask * cell_mask
# Expand the nucleus_mask to match the shape of the other masks
nucleus_mask_expanded = np.repeat(avg_nucleus[np.newaxis, :, :], cell_mask.shape[0], axis=0)

# Remove vesicles from the nucleus in each frame
VLS_in_cytoplasm = VLS_in_cell & ~nucleus_mask_expanded 
                     
io.imsave(filename + '_binary_vesicles.tif', VLS_in_cytoplasm, check_contrast=False)

## Final Clean-up: Remove VLS and nucleus from cell mask 

To obtain a clean image of the cytoplasm, devoid of VLS and out the nucleus, we set all pixels which are "TRUE" in either nucleus or VLS to "FALSE" in the cell mask.

In [17]:
# Create the cytoplasm mask by removing the nucleus and vesicles from the total cell
cytoplasm_mask = cell_mask & ~nucleus_mask_expanded & ~VLS_in_cytoplasm

# save result
io.imsave(filename + '_wo nucleus_vesicles.tif', cytoplasm_mask, check_contrast=False)

## Display masks for 1st frame

In [2]:
plt.figure(figsize=(12,12))
plt.subplot(141)
plt.imshow(img[0], cmap="inferno")
plt.title("Original Image")
plt.subplot(142)
plt.imshow(cytoplasm_mask[0])
plt.title("Cytoplasm")
plt.subplot(143)
plt.imshow(avg_nucleus)
plt.title("Average nucleus")
plt.subplot(144)
plt.imshow(VLS_in_cytoplasm[0])
plt.title("Vesicle-like structures")
plt.tight_layout()

NameError: name 'plt' is not defined